In [159]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [35]:
df = pd.read_csv('https://docs.yandex.ru/docs/view?url=ya-disk-public%3A%2F%2FjwCWnJsZOJ4kEoVdSjLB%2Blh2ohp9VBHsliS8CYBBDIIVAzwdujVb338UTiJHEitiq%2FJ6bpmRyOJonT3VoXnDag%3D%3D%3A%2Ftrain.csv&name=train.csv&nosw=1')
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [36]:
df1 = df.drop(['id','timestamp','sub_area'], axis= 1 )
df1.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,

In [37]:
cleanup_nums = {"product_type":     {"Investment": 0, "OwnerOccupier": 1},
                "culture_objects_top_25": {"no": 0, "yes": 1 },
               "thermal_power_plant_raion": {"no": 0, "yes": 1 },
               "incineration_raion": {"no": 0, "yes": 1 },
                "oil_chemistry_raion": {"no": 0, "yes": 1 },
                "radiation_raion": {"no": 0, "yes": 1 },
                "railroad_terminal_raion": {"no": 0, "yes": 1 },
                "big_market_raion": {"no": 0, "yes": 1 },
                "nuclear_reactor_raion": {"no": 0, "yes": 1 },
                "detention_facility_raion": {"no": 0, "yes": 1 },
                "water_1line": {"no": 0, "yes": 1 },
                "big_road1_1line": {"no": 0, "yes": 1 },
                "railroad_1line": {"no": 0, "yes": 1 },
                "ecology": {"poor": 0, "no data": 1, "good": 2, "excellent": 3, "satisfactory": 4}
               }
df1 = df1.replace(cleanup_nums)
df1.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,

In [13]:
columns_ = ['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_quota', 'preschool_education_centers_raion', 'children_school', 'school_quota', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'hospital_beds_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_monolith', 'build_count_panel', 'build_count_foam', 'build_count_slag', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_before_1920', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995', 'ID_metro', 'metro_min_avto', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk', 'kindergarten_km', 'school_km', 'park_km', 'green_zone_km', 'industrial_km', 'water_treatment_km', 'cemetery_km', 'incineration_km', 'railroad_station_walk_km', 'railroad_station_walk_min', 'ID_railroad_station_walk', 'railroad_station_avto_km', 'railroad_station_avto_min', 'ID_railroad_station_avto', 'public_transport_station_km', 'public_transport_station_min_walk', 'water_km', 'water_1line', 'mkad_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road1_1line', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'railroad_1line', 'zd_vokzaly_avto_km', 'ID_railroad_terminal', 'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km', 'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km', 'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km', 'fitness_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km', 'basketball_km', 'hospice_morgue_km', 'detention_facility_km', 'public_healthcare_km', 'university_km', 'workplaces_km', 'shopping_centers_km', 'office_km', 'additional_education_km', 'preschool_km', 'big_church_km', 'church_synagogue_km', 'mosque_km', 'theater_km', 'museum_km', 'exhibition_km', 'catering_km', 'ecology', 'green_part_500', 'prom_part_500', 'office_count_500', 'office_sqm_500', 'trc_count_500', 'trc_sqm_500', 'cafe_count_500', 'cafe_sum_500_min_price_avg', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_count_500_na_price', 'cafe_count_500_price_500', 'cafe_count_500_price_1000', 'cafe_count_500_price_1500', 'cafe_count_500_price_2500', 'cafe_count_500_price_4000', 'cafe_count_500_price_high', 'big_church_count_500', 'church_count_500', 'mosque_count_500', 'leisure_count_500', 'sport_count_500', 'market_count_500', 'green_part_1000', 'prom_part_1000', 'office_count_1000', 'office_sqm_1000', 'trc_count_1000', 'trc_sqm_1000', 'cafe_count_1000', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_count_1000_na_price', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000', 'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'cafe_count_1000_price_high', 'big_church_count_1000', 'church_count_1000', 'mosque_count_1000', 'leisure_count_1000', 'sport_count_1000', 'market_count_1000', 'green_part_1500', 'prom_part_1500', 'office_count_1500', 'office_sqm_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_count_1500', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_count_1500_na_price', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500', 'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'cafe_count_1500_price_high', 'big_church_count_1500', 'church_count_1500', 'mosque_count_1500', 'leisure_count_1500', 'sport_count_1500', 'market_count_1500', 'green_part_2000', 'prom_part_2000', 'office_count_2000', 'office_sqm_2000', 'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'cafe_count_2000_na_price', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500', 'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'big_church_count_2000', 'church_count_2000', 'mosque_count_2000', 'leisure_count_2000', 'sport_count_2000', 'market_count_2000', 'green_part_3000', 'prom_part_3000', 'office_count_3000', 'office_sqm_3000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_count_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'cafe_count_3000_na_price', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'big_church_count_3000', 'church_count_3000', 'mosque_count_3000', 'leisure_count_3000', 'sport_count_3000', 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'office_count_5000', 'office_sqm_5000', 'trc_count_5000', 'trc_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'mosque_count_5000', 'leisure_count_5000', 'sport_count_5000', 'market_count_5000', 'price_doc']

In [43]:
s = []
for i in df1['price_doc']:
    if i < 4.740002e+06: s.append(0)
    elif 4.740002e+06 <= i < 6.274411e+06: s.append(1)
    elif 6.274411e+06 <= i < 8.300000e+06: s.append(2)
    else: s.append(3)
df1['price_doc1'] = s
df1 = df1.drop(['price_doc'], axis= 1 )

In [38]:
for i in columns_:
    df1[i] = df1[i].fillna(df1[i].mean())

In [96]:
df1.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,

In [134]:
columns_target = ["price_doc1"] #целевая колонка
columns_train = ['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_quota', 'preschool_education_centers_raion', 'children_school', 'school_quota', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'hospital_beds_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_monolith', 'build_count_panel', 'build_count_foam', 'build_count_slag', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_before_1920', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995', 'ID_metro', 'metro_min_avto', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk', 'kindergarten_km', 'school_km', 'park_km', 'green_zone_km', 'industrial_km', 'water_treatment_km', 'cemetery_km', 'incineration_km', 'railroad_station_walk_km', 'railroad_station_walk_min', 'ID_railroad_station_walk', 'railroad_station_avto_km', 'railroad_station_avto_min', 'ID_railroad_station_avto', 'public_transport_station_km', 'public_transport_station_min_walk', 'water_km', 'water_1line', 'mkad_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road1_1line', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'railroad_1line', 'zd_vokzaly_avto_km', 'ID_railroad_terminal', 'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km', 'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km', 'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km', 'fitness_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km', 'basketball_km', 'hospice_morgue_km', 'detention_facility_km', 'public_healthcare_km', 'university_km', 'workplaces_km', 'shopping_centers_km', 'office_km', 'additional_education_km', 'preschool_km', 'big_church_km', 'church_synagogue_km', 'mosque_km', 'theater_km', 'museum_km', 'exhibition_km', 'catering_km', 'ecology', 'green_part_500', 'prom_part_500', 'office_count_500', 'office_sqm_500', 'trc_count_500', 'trc_sqm_500', 'cafe_count_500', 'cafe_sum_500_min_price_avg', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_count_500_na_price', 'cafe_count_500_price_500', 'cafe_count_500_price_1000', 'cafe_count_500_price_1500', 'cafe_count_500_price_2500', 'cafe_count_500_price_4000', 'cafe_count_500_price_high', 'big_church_count_500', 'church_count_500', 'mosque_count_500', 'leisure_count_500', 'sport_count_500', 'market_count_500', 'green_part_1000', 'prom_part_1000', 'office_count_1000', 'office_sqm_1000', 'trc_count_1000', 'trc_sqm_1000', 'cafe_count_1000', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_count_1000_na_price', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000', 'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'cafe_count_1000_price_high', 'big_church_count_1000', 'church_count_1000', 'mosque_count_1000', 'leisure_count_1000', 'sport_count_1000', 'market_count_1000', 'green_part_1500', 'prom_part_1500', 'office_count_1500', 'office_sqm_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_count_1500', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_count_1500_na_price', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500', 'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'cafe_count_1500_price_high', 'big_church_count_1500', 'church_count_1500', 'mosque_count_1500', 'leisure_count_1500', 'sport_count_1500', 'market_count_1500', 'green_part_2000', 'prom_part_2000', 'office_count_2000', 'office_sqm_2000', 'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'cafe_count_2000_na_price', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500', 'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'big_church_count_2000', 'church_count_2000', 'mosque_count_2000', 'leisure_count_2000', 'sport_count_2000', 'market_count_2000', 'green_part_3000', 'prom_part_3000', 'office_count_3000', 'office_sqm_3000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_count_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'cafe_count_3000_na_price', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'big_church_count_3000', 'church_count_3000', 'mosque_count_3000', 'leisure_count_3000', 'sport_count_3000', 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'office_count_5000', 'office_sqm_5000', 'trc_count_5000', 'trc_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'mosque_count_5000', 'leisure_count_5000', 'sport_count_5000', 'market_count_5000']

In [135]:
X = df1[columns_train]
Y = df1[columns_target]

In [136]:
chi2_features = SelectKBest(chi2, k = 28)
X_kbest_features = chi2_features.fit_transform(X, Y)

# Вывод "до и после"
print("Количество признаков до преобразования:", X.shape[1])
print("Количество признаков после преобразования:", X_kbest_features.shape[1])

Количество признаков до преобразования: 288
Количество признаков после преобразования: 28


In [143]:
importances = mutual_info_classif(X, Y)
feature_importances = pd.Series(importances, df1.columns[0:len(df1.columns)-1])
print(feature_importances.sort_values(ascending=False)[:28])

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


kremlin_km                    0.220487
full_sq                       0.219977
bulvar_ring_km                0.218851
trc_sqm_5000                  0.216772
incineration_km               0.216556
oil_chemistry_km              0.215964
zd_vokzaly_avto_km            0.215589
sadovoe_km                    0.214114
ttk_km                        0.212948
detention_facility_km         0.212901
swim_pool_km                  0.212860
metro_km_walk                 0.212749
office_km                     0.212707
mkad_km                       0.211445
school_km                     0.210937
metro_min_walk                0.210898
big_church_km                 0.210704
public_healthcare_km          0.210573
park_km                       0.209571
catering_km                   0.209388
cafe_avg_price_5000           0.209198
basketball_km                 0.208914
nuclear_reactor_km            0.208905
university_km                 0.208790
stadium_km                    0.208628
power_transmission_line_k

In [146]:
columns_train = ['kremlin_km', 'full_sq', 'bulvar_ring_km', 'trc_sqm_5000', 'incineration_km', 'oil_chemistry_km', 'zd_vokzaly_avto_km', 'sadovoe_km', 'ttk_km', 'detention_facility_km', 'swim_pool_km', 'metro_km_walk', 'office_km', 'mkad_km', 'school_km', 'metro_min_walk', 'big_church_km', 'public_healthcare_km', 'park_km', 'catering_km', 'cafe_avg_price_5000', 'basketball_km', 'nuclear_reactor_km', 'university_km', 'stadium_km', 'power_transmission_line_km', 'theater_km', 'metro_min_avto']
Y = df1[columns_target]

In [171]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size =0.3, random_state=22)

In [172]:
predmodel = CatBoostClassifier()
predmodel.fit(X_train, Y_train)

Learning rate set to 0.092491
0:	learn: 1.3159351	total: 60.1ms	remaining: 1m
1:	learn: 1.2597505	total: 111ms	remaining: 55.2s
2:	learn: 1.2137023	total: 159ms	remaining: 52.8s
3:	learn: 1.1777781	total: 207ms	remaining: 51.6s
4:	learn: 1.1460524	total: 255ms	remaining: 50.8s
5:	learn: 1.1186003	total: 302ms	remaining: 50s
6:	learn: 1.0921592	total: 350ms	remaining: 49.6s
7:	learn: 1.0674414	total: 399ms	remaining: 49.5s
8:	learn: 1.0491283	total: 449ms	remaining: 49.5s
9:	learn: 1.0289203	total: 494ms	remaining: 48.9s
10:	learn: 1.0133712	total: 550ms	remaining: 49.5s
11:	learn: 0.9971492	total: 600ms	remaining: 49.4s
12:	learn: 0.9835317	total: 657ms	remaining: 49.9s
13:	learn: 0.9712407	total: 708ms	remaining: 49.9s
14:	learn: 0.9600636	total: 758ms	remaining: 49.8s
15:	learn: 0.9493287	total: 812ms	remaining: 49.9s
16:	learn: 0.9397870	total: 861ms	remaining: 49.8s
17:	learn: 0.9302929	total: 912ms	remaining: 49.8s
18:	learn: 0.9229701	total: 965ms	remaining: 49.8s
19:	learn: 0.91

162:	learn: 0.6984522	total: 8.03s	remaining: 41.3s
163:	learn: 0.6978075	total: 8.08s	remaining: 41.2s
164:	learn: 0.6971403	total: 8.13s	remaining: 41.2s
165:	learn: 0.6965427	total: 8.18s	remaining: 41.1s
166:	learn: 0.6962742	total: 8.22s	remaining: 41s
167:	learn: 0.6957170	total: 8.27s	remaining: 41s
168:	learn: 0.6952808	total: 8.32s	remaining: 40.9s
169:	learn: 0.6939019	total: 8.37s	remaining: 40.9s
170:	learn: 0.6932474	total: 8.42s	remaining: 40.8s
171:	learn: 0.6929946	total: 8.47s	remaining: 40.8s
172:	learn: 0.6925690	total: 8.53s	remaining: 40.8s
173:	learn: 0.6921388	total: 8.58s	remaining: 40.7s
174:	learn: 0.6914336	total: 8.63s	remaining: 40.7s
175:	learn: 0.6910095	total: 8.68s	remaining: 40.6s
176:	learn: 0.6904338	total: 8.74s	remaining: 40.6s
177:	learn: 0.6902371	total: 8.78s	remaining: 40.5s
178:	learn: 0.6895456	total: 8.83s	remaining: 40.5s
179:	learn: 0.6891518	total: 8.88s	remaining: 40.5s
180:	learn: 0.6886269	total: 8.93s	remaining: 40.4s
181:	learn: 0.68

321:	learn: 0.6363590	total: 15.8s	remaining: 33.2s
322:	learn: 0.6360237	total: 15.8s	remaining: 33.1s
323:	learn: 0.6350924	total: 15.8s	remaining: 33.1s
324:	learn: 0.6348278	total: 15.9s	remaining: 33s
325:	learn: 0.6345276	total: 15.9s	remaining: 33s
326:	learn: 0.6343040	total: 16s	remaining: 32.9s
327:	learn: 0.6341164	total: 16s	remaining: 32.8s
328:	learn: 0.6339233	total: 16.1s	remaining: 32.8s
329:	learn: 0.6333865	total: 16.1s	remaining: 32.7s
330:	learn: 0.6329151	total: 16.2s	remaining: 32.7s
331:	learn: 0.6328298	total: 16.2s	remaining: 32.6s
332:	learn: 0.6326085	total: 16.3s	remaining: 32.6s
333:	learn: 0.6323475	total: 16.3s	remaining: 32.6s
334:	learn: 0.6319482	total: 16.4s	remaining: 32.5s
335:	learn: 0.6317531	total: 16.4s	remaining: 32.5s
336:	learn: 0.6315537	total: 16.5s	remaining: 32.4s
337:	learn: 0.6312506	total: 16.5s	remaining: 32.3s
338:	learn: 0.6311122	total: 16.6s	remaining: 32.3s
339:	learn: 0.6307974	total: 16.6s	remaining: 32.2s
340:	learn: 0.630703

483:	learn: 0.5928347	total: 23.6s	remaining: 25.1s
484:	learn: 0.5924439	total: 23.6s	remaining: 25.1s
485:	learn: 0.5922713	total: 23.7s	remaining: 25s
486:	learn: 0.5920842	total: 23.7s	remaining: 25s
487:	learn: 0.5918137	total: 23.8s	remaining: 24.9s
488:	learn: 0.5914943	total: 23.8s	remaining: 24.9s
489:	learn: 0.5911027	total: 23.9s	remaining: 24.9s
490:	learn: 0.5909300	total: 23.9s	remaining: 24.8s
491:	learn: 0.5908086	total: 24s	remaining: 24.7s
492:	learn: 0.5905723	total: 24s	remaining: 24.7s
493:	learn: 0.5904646	total: 24.1s	remaining: 24.6s
494:	learn: 0.5902668	total: 24.1s	remaining: 24.6s
495:	learn: 0.5902411	total: 24.2s	remaining: 24.5s
496:	learn: 0.5898371	total: 24.2s	remaining: 24.5s
497:	learn: 0.5894982	total: 24.3s	remaining: 24.5s
498:	learn: 0.5892967	total: 24.3s	remaining: 24.4s
499:	learn: 0.5890752	total: 24.4s	remaining: 24.4s
500:	learn: 0.5887864	total: 24.4s	remaining: 24.3s
501:	learn: 0.5885439	total: 24.4s	remaining: 24.3s
502:	learn: 0.588335

644:	learn: 0.5579310	total: 31.4s	remaining: 17.3s
645:	learn: 0.5577939	total: 31.5s	remaining: 17.3s
646:	learn: 0.5576189	total: 31.5s	remaining: 17.2s
647:	learn: 0.5572533	total: 31.6s	remaining: 17.2s
648:	learn: 0.5571956	total: 31.6s	remaining: 17.1s
649:	learn: 0.5570520	total: 31.7s	remaining: 17.1s
650:	learn: 0.5567795	total: 31.7s	remaining: 17s
651:	learn: 0.5565212	total: 31.8s	remaining: 17s
652:	learn: 0.5562628	total: 31.8s	remaining: 16.9s
653:	learn: 0.5561637	total: 31.9s	remaining: 16.9s
654:	learn: 0.5558153	total: 31.9s	remaining: 16.8s
655:	learn: 0.5556331	total: 32s	remaining: 16.8s
656:	learn: 0.5552704	total: 32s	remaining: 16.7s
657:	learn: 0.5549985	total: 32.1s	remaining: 16.7s
658:	learn: 0.5548599	total: 32.1s	remaining: 16.6s
659:	learn: 0.5547071	total: 32.2s	remaining: 16.6s
660:	learn: 0.5544570	total: 32.2s	remaining: 16.5s
661:	learn: 0.5543184	total: 32.3s	remaining: 16.5s
662:	learn: 0.5541562	total: 32.3s	remaining: 16.4s
663:	learn: 0.553951

804:	learn: 0.5284900	total: 39.2s	remaining: 9.5s
805:	learn: 0.5282025	total: 39.3s	remaining: 9.45s
806:	learn: 0.5281049	total: 39.3s	remaining: 9.4s
807:	learn: 0.5280641	total: 39.4s	remaining: 9.35s
808:	learn: 0.5279182	total: 39.4s	remaining: 9.3s
809:	learn: 0.5277499	total: 39.5s	remaining: 9.25s
810:	learn: 0.5275460	total: 39.5s	remaining: 9.21s
811:	learn: 0.5273865	total: 39.6s	remaining: 9.16s
812:	learn: 0.5271244	total: 39.6s	remaining: 9.11s
813:	learn: 0.5269234	total: 39.7s	remaining: 9.06s
814:	learn: 0.5266497	total: 39.7s	remaining: 9.01s
815:	learn: 0.5263445	total: 39.8s	remaining: 8.96s
816:	learn: 0.5260814	total: 39.8s	remaining: 8.92s
817:	learn: 0.5259970	total: 39.9s	remaining: 8.87s
818:	learn: 0.5257188	total: 39.9s	remaining: 8.82s
819:	learn: 0.5256338	total: 40s	remaining: 8.77s
820:	learn: 0.5255031	total: 40s	remaining: 8.72s
821:	learn: 0.5253201	total: 40.1s	remaining: 8.67s
822:	learn: 0.5250764	total: 40.1s	remaining: 8.63s
823:	learn: 0.52497

963:	learn: 0.5023586	total: 47s	remaining: 1.75s
964:	learn: 0.5022663	total: 47s	remaining: 1.7s
965:	learn: 0.5022111	total: 47s	remaining: 1.66s
966:	learn: 0.5019648	total: 47.1s	remaining: 1.61s
967:	learn: 0.5017107	total: 47.1s	remaining: 1.56s
968:	learn: 0.5015356	total: 47.2s	remaining: 1.51s
969:	learn: 0.5012881	total: 47.2s	remaining: 1.46s
970:	learn: 0.5010210	total: 47.3s	remaining: 1.41s
971:	learn: 0.5007853	total: 47.3s	remaining: 1.36s
972:	learn: 0.5007367	total: 47.4s	remaining: 1.31s
973:	learn: 0.5006508	total: 47.4s	remaining: 1.27s
974:	learn: 0.5005450	total: 47.5s	remaining: 1.22s
975:	learn: 0.5003575	total: 47.5s	remaining: 1.17s
976:	learn: 0.5001698	total: 47.6s	remaining: 1.12s
977:	learn: 0.5000338	total: 47.6s	remaining: 1.07s
978:	learn: 0.4999008	total: 47.7s	remaining: 1.02s
979:	learn: 0.4997799	total: 47.7s	remaining: 974ms
980:	learn: 0.4997046	total: 47.8s	remaining: 925ms
981:	learn: 0.4995590	total: 47.8s	remaining: 877ms
982:	learn: 0.49941

In [173]:
predmodel = predmodel.predict(X_test)

In [176]:
classification_report(Y_test,predmodel, labels=[0, 1, 2, 3])

'              precision    recall  f1-score   support\n\n           0       0.81      0.59      0.68      2271\n           1       0.70      0.73      0.72      2321\n           2       0.65      0.73      0.69      2295\n           3       0.79      0.87      0.82      2255\n\n    accuracy                           0.73      9142\n   macro avg       0.74      0.73      0.73      9142\nweighted avg       0.74      0.73      0.73      9142\n'